In [ ]:
# Step 1: Import required modules (make sure to pip install -r requirements.txt before this)
import pinecone
from google.colab import files
import os
import shutil
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Pinecone
from langchain_pinecone import Pinecone  # Correct import
from langchain.embeddings.openai import OpenAIEmbeddings  # Use OpenAI embeddings or SentenceTransformer

In [ ]:
from google.colab import files

# Uploading file
uploaded = files.upload()

Saving proposal.pdf to proposal (5).pdf


In [ ]:
# Get the file path of the uploaded PDF
pdf_path = list(uploaded.keys())[0]

In [ ]:
# Step 2: Extract text from the uploaded PDF file
import PyPDF2

def extract_text_from_pdf(pdf_path):
    reader = PyPDF2.PdfReader(pdf_path)
    text = ''
    for page_num in range(len(reader.pages)):
        text += reader.pages[page_num].extract_text()
    return text

In [ ]:
# Extract text from the PDF
documents = extract_text_from_pdf(pdf_path)

In [ ]:
# Step 3: Split documents into chunks
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Convert loaded documents into Document objects if they are strings
def convert_to_document_objects(documents):
    doc_objects = []
    for doc in documents:
        if isinstance(doc, str):
            doc_objects.append(Document(page_content=doc))  # Ensure documents are converted to Document objects
        else:
            doc_objects.append(doc)
    return doc_objects

# Split documents into smaller chunks
def split_docs(documents, chunk_size=500, chunk_overlap=20):  # Adjusted sizes for better context
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

# Ensure that the input document is valid
if isinstance(documents, str):
    documents = [documents]  # Ensure it's a list of documents

# Convert documents to objects with page_content
doc_objects = convert_to_document_objects(documents)  # Use doc_objects instead of raw strings

# Check for the conversion result
print(f"Converted {len(doc_objects)} documents to Document objects.")

# Now split the documents
docs = split_docs(doc_objects)  # Pass doc_objects to split_docs

# Check result
print(f"Total chunks created: {len(docs)}")
for i in range(min(5, len(docs))):  # Display the first 5 chunks to check the result
    print(f"Chunk {i + 1}:\n{docs[i].page_content}\n")

Converted 1 documents to Document objects.
Total chunks created: 54
Chunk 1:
1 
 
“Study Of AI Strategies of Selected Organizations And Their Societal 
Impact With Special Reference  To India " 
 
1) Introduction  
 
Artificial Intelligence (AI)  
Today's age is the age of Artificial Intelligence. AI has become 
increasingly prevalent across industries and sectors due to rapid 
advancements in computing power, algorithms, and big data. AI refers 
to computer systems that can perform tasks that typically require

Chunk 2:
human intelligence, such as visual perception, speech recognition, 
decision -making, a nd language translation.  
 
The tech sector plays a key role in the development and deployment 
of AI technologies. In technologically advanced countries like the 
United States, AI is driving innovation and economic growth. Due to 
the rapid progress in machin e learning, neural networks, and other AI 
techniques, many industries are undergoing an AI revolution.

Chunk 3:
Dramatic

In [ ]:
# Step 4: Initialize the embeddings model
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-9-2f01879b9e9f>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

In [ ]:
# Step 5: Initialize Pinecone
from pinecone import Pinecone

# Initialize Pinecone using your API key
pc = Pinecone(
    api_key="d79317b9-680b-4166-8bb4-4cb913892719"  # Replace with your actual API key
)

# Define the index name you already created
index_name = "langchain-chatbot"  # Ensure this matches the name of the index in the Pinecone UI

# Access the existing index
index = pc.Index(index_name)

# Ensure you have initialized the index before inserting vectors

# Step 6: Generate embeddings for the documents and insert them into Pinecone
doc_texts = [doc.page_content for doc in docs]  # Extract the text from each document chunk
doc_embeddings = embeddings.embed_documents(doc_texts)  # Generate embeddings for each document chunk


# Prepare metadata (optional)
metadata = [{"id": str(i), "text": doc.page_content} for i, doc in enumerate(docs)]  # Generate metadata for each document

# Debugging: Check lengths
print(f"Number of document embeddings: {len(doc_embeddings)}")
print(f"Number of metadata entries: {len(metadata)}")

# Ensure both lengths match
if len(doc_embeddings) != len(metadata):
    print("Warning: Lengths of embeddings and metadata do not match!")

# Prepare vectors with IDs and metadata
vectors = []
for i in range(len(doc_embeddings)):
    # Ensure you are within bounds
    if i < len(metadata):
        vectors.append((str(i), doc_embeddings[i], metadata[i]))
    else:
        print(f"Skipping index {i} due to missing metadata.")

# Function to batch the vectors into smaller chunks
def batch_vectors(vectors, batch_size=100):
    for i in range(0, len(vectors), batch_size):
        yield vectors[i:i + batch_size]

# Upsert vectors into Pinecone in batches
batch_size = 100  # Adjust based on the size of your vectors and Pinecone limits
for batch in batch_vectors(vectors, batch_size=batch_size):
    index.upsert(vectors=batch)  # Insert vectors in batches

print(f"Successfully inserted {len(vectors)} vectors into Pinecone.")

Number of document embeddings: 54
Number of metadata entries: 54
Successfully inserted 54 vectors into Pinecone.


In [ ]:
# Step 7: Function to search for similar documents
def get_similar_docs(query, k=5, score=False):
    # Generate query embeddings
    query_embedding = embeddings.embed_query(query)  # Ensure it's a single string

    # Perform similarity search with or without scores
    if score:
        similar_docs = index.query(vector=query_embedding, top_k=k, include_metadata=True)  # Use keyword arguments
    else:
        similar_docs = index.query(vector=query_embedding, top_k=k)

    return similar_docs

# Example query
query = "will ai have any impacts on jobs?"
results = get_similar_docs(query, k=5, score=True)  # Adjust k to retrieve more documents

# Display the results
for match in results['matches']:
    # Check for the attributes present in match
    match_id = match.get('id', 'No ID')
    match_score = match.get('score', 'No Score')
    match_metadata = match.get('metadata', {}).get('text', 'No Text')  # Use get to avoid KeyError

    print(f"ID: {match_id}, Score: {match_score}, Text: {match_metadata}")

ID: 33, Score: 0.719475925, Text: amplifying existing societal challenges rather th an solving them.  
 
In summary, the literature reveals a spectrum of views on whether AI 
is good or bad. While some emphasize its potential for societal benefit 
and innovation, others caution against its risks, such as job 
displacement, bias, and loss of con trol. Most scholars agree, however, 
that the impact of AI will largely depend on how it is governed and 
whether ethical principles are incorporated into its development and
ID: 15, Score: 0.670440793, Text: 1) Job displacement due to automation of routine cognitive and 
manual tasks  
2) Exacerbation of economic inequality if benefits of AI accrue mainly 
to skilled workers and capital owners  
3) Algorithmic  bias leading to unfair outcomes for certain groups  
4) Privacy and data protection risks from AI systems that process 
personal data  6 
 
5) Safety risks from autonomous AI systems in critical domains  
6) Potential for AI to be misuse

In [ ]:
pip freeze > requirements.txt